## Exploring the Dataset

### Loading the Dataset

In [ ]:
# Upload kaggle config file
from google.colab import files
files.upload()

In [ ]:
# Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the dataset
!kaggle datasets download -d xhlulu/recursion-cellular-image-classification-224-jpg

In [ ]:
# Extract the data
!mkdir /content/recursion-cellular-image-classification
!unzip /content/recursion-cellular-image-classification-224-jpg.zip -d /content/recursion-cellular-image-classification

## Code

In [ ]:
import numpy as np
import pandas as pd
import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn.model_selection import train_test_split

In [ ]:
# Définir le chemin de dataset
base_path = "/content/recursion-cellular-image-classification/"

# Lire le ficher "new_train.csv"
df = pd.read_csv(base_path + "new_train.csv")

# Définir le nombre des classes
num_classes = 100

df = df[df.sirna < num_classes]

# Preprocess les images
def preprocess_images(df):
    images = []
    for i, row in df.iterrows():
        path = base_path + "train/train/" + row['filename']
        img = cv2.imread(path)
        img = cv2.resize(img, (224, 224))        
        images.append(img)
    return np.array(images)

X = preprocess_images(df)
y = to_categorical(df['sirna'], num_classes=num_classes)

In [ ]:
# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(
    X,  # the data
    y,  # the labels
    test_size=0.2,  # the proportion of the data to use for testing
)

# Print the shape of the training and testing sets
print('Training data shape:', train_data.shape)
print('Training labels shape:', train_labels.shape)
print('Testing data shape:', test_data.shape)
print('Testing labels shape:', test_labels.shape)

Training data shape: (5273, 224, 224, 3)
Training labels shape: (5273, 100)
Testing data shape: (1319, 224, 224, 3)
Testing labels shape: (1319, 100)


### Simple model

In [ ]:
input_shape = (224, 224, 3)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=input_shape, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

### Training

In [ ]:
epochs = 5
batch_size = 128

history = model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/5
38/38 [==============================] - 448s 12s/step - loss: 57.1388 - accuracy: 0.0080 - val_loss: 5.4751 - val_accuracy: 0.0076
Epoch 2/5
38/38 [==============================] - 484s 13s/step - loss: 5.4630 - accuracy: 0.0124 - val_loss: 5.4472 - val_accuracy: 0.0038
Epoch 3/5
38/38 [==============================] - 479s 13s/step - loss: 5.4122 - accuracy: 0.0124 - val_loss: 5.4052 - val_accuracy: 0.0076
Epoch 4/5
38/38 [==============================] - 482s 13s/step - loss: 5.3511 - accuracy: 0.0126 - val_loss: 5.3627 - val_accuracy: 0.0095
Epoch 5/5
38/38 [==============================] - 480s 13s/step - loss: 5.3076 - accuracy: 0.0139 - val_loss: 5.3354 - val_accuracy: 0.0076


### with RestNet50

In [ ]:
# Load the ResNet50 model
resnet = ResNet50(include_top=False, input_shape=(224, 224, 3), weights='imagenet')

# Add a few more layers for classification
model = tf.keras.Sequential([
    resnet,
    Flatten(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.layers[0].trainable = False


# Compile the model
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
epochs = 5
batch_size = 128

history = model.fit(train_data, train_labels, batch_size=batch_size, epochs=epochs, validation_split=0.1)